# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [243]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Check basic VGG19 Classification

In [244]:
testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# Convert Image pixels to a numpy array to be operated on
pictureArray = img_to_array(testPicture)
# reshape Data for model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# predict the probability across all output classes
yhat = preTrainedModel.predict(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)



2023-10-25 11:50:44.829648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 541ms/step
[[('n02441942', 'weasel', 0.3109628), ('n02443484', 'black-footed_ferret', 0.17763011), ('n02443114', 'polecat', 0.17633283), ('n02447366', 'badger', 0.16493277), ('n02442845', 'mink', 0.059160564)]]


## Create complete Model with Custom output Layer

In [245]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(75, activation = "relu"))
completeModel.add(Dense(50, activation = "relu"))
completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(5, activation = "softmax"))
completeModel.summary()
completeModel.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_41 (Dense)            (None, 100)               100100    
                                                                 
 dense_42 (Dense)            (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data

In [246]:
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels="inferred",
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

Found 2251 files belonging to 2 classes.
Found 314 files belonging to 2 classes.


## Prediction before training

In [247]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-10-25 11:50:45.652401: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 511ms/step
[[0.5159187]]


## Train Last two layers

In [248]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


ValueError: in user code:

    File "/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.co

## Test Some data

In [ ]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

2023-10-25 11:40:37.434356: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 2s 353ms/step - loss: 0.0045 - accuracy: 1.0000
loss =  0.0044530280865728855
accuracy =  1.0


## See what happens with a deer image

In [ ]:
from PIL import Image
testPicture = load_img('testData/testDeer/SEQ86955_IMG_0001.JPG___crop00_md_v5a.0.0.pt.jpg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 88ms/step
[[0.99582046]]


## Save the model for export

In [ ]:
# completeModel.save("multiModel.keras")

## Check test data for each animal

In [ ]:
# import required module
import os
# assign directory
directory = 'testData/testDeer'
 
# iterate over files in
# that directory
results = [0,0, 0, 0, 0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = max(zhat[0])
        results[zhat[0].index(x)] += 1
print (results)

testData/testDeer/SEQ81957_IMG_0003.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 86ms/step
[[0.9950465]]
testData/testDeer/SEQ80916_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.99516135]]
testData/testDeer/SEQ86955_IMG_0001.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 46ms/step
[[0.99582046]]
testData/testDeer/SEQ87558_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9946924]]
testData/testDeer/SEQ81957_IMG_0010.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.9950918]]
testData/testDeer/SEQ86931_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9952514]]
testData/testDeer/SEQ81957_IMG_0016.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.99524164]]
testData/testDeer/SEQ87611_IMG_0006.JPG___crop01_md_v5a.0.0.pt.jpg

In [ ]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = max(zhat[0])
        results[zhat[0].index(x)] += 1
print (results)

extraTest/testFox/SEQ79862_IMG_0012.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 23ms/step
[[0.99540555]]
extraTest/testFox/SEQ80191_IMG_0002.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9944502]]
extraTest/testFox/SEQ79862_IMG_0003.JPG___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9951652]]
extraTest/testFox/SEQ81302_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9952108]]
extraTest/testFox/SEQ79869_IMG_0008.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9958085]]
extraTest/testFox/SEQ80191_IMG_0010.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99616975]]
extraTest/testFox/SEQ79868_IMG_0002.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [========

KeyboardInterrupt: 

In [ ]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = max(zhat[0])
        results[zhat[0].index(x)] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/2005.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/2016.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/2059.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/2123.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/2130.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossum/1937.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
extraTest/testPossu

KeyboardInterrupt: 

In [ ]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0,0,0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = max(zhat[0])
        results[zhat[0].index(x)] += 1
print (results)

testData/testRabbit/791.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testRabbit/782.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
testData/testRabbit/821.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
testData/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
testData/testRabbit/832.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
testData/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testRabbit/784.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
testData/testRabbit/797.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testRabbit/834.jpg___crop00_

In [ ]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = max(zhat[0])
        results[zhat[0].index(x)] += 1
print (results)

testData/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 29ms/step
testData/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testSquirrel/91.jpg___crop04_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testSquirrel/96.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
testData/testSquirrel/82.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
testData/testSquirrel/84.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
testData/testSquirrel/88.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
testData/testSquirrel/90.jpg

## Code for data manipulation to creaet more data

In [ ]:
# directory = 'trainingData/trainRabbit' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])